In [7]:
from os.path import abspath, dirname, exists, join
repository_folder = dirname(dirname(abspath('.')))
configuration_path = join(repository_folder, 'development.ini')
assert exists(configuration_path)

In [8]:
from pyramid.paster import bootstrap
env = bootstrap(configuration_path)
env

{'root': <pyramid.traversal.DefaultRootFactory at 0x7fc229d2ad50>,
 'closer': <function pyramid.scripting.prepare.<locals>.closer()>,
 'registry': <Registry asset_tracker>,
 'request': <Request at 0x7fc229d2af10 GET http://localhost/>,
 'root_factory': pyramid.traversal.DefaultRootFactory,
 'app': <pyramid.router.Router at 0x7fc256289d90>}

In [ ]:
# Get database connection
# request = env['request']
# db = request.db
# db

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

database_engine = create_engine('sqlite://', echo=True)
get_database_session = sessionmaker()
get_database_session.configure(bind=database_engine)
db = get_database_session()

In [ ]:
from asset_tracker.models.meta import Base
Base.metadata.create_all(database_engine)

In [ ]:
# Load JSON
import json
datasets_folder = join(repository_folder, 'datasets')
assets_json_path = join(datasets_folder, 'assets1.json')
assets_geojson_path = join(datasets_folder, 'assets1.geojson')

In [ ]:
asset_dictionaries = json.load(open(assets_json_path, 'rt'))
asset_feature_collection = json.load(open(assets_geojson_path, 'rt'))

In [ ]:
asset_feature_collection.keys()

In [ ]:
asset_dictionaries

In [ ]:
from asset_tracker.models import *

In [ ]:
# Print asset_id for each asset
for asset_d in asset_dictionaries:
    print(asset_d['id'])

In [ ]:
db

In [ ]:
import transaction

with transaction.manager:
    print(db.query(Asset).count())

In [ ]:
# Check if asset id exists in database for each asset
for asset_d in asset_dictionaries:
    asset_id = asset_d['id']
    asset = db.query(Asset).first()
    print(asset)

In [ ]:
db.query(Asset).count()

In [ ]:
asset_dictionaries[0]

In [ ]:
asset_dictionaries[1]

In [ ]:
print(len(asset_dictionaries))

In [ ]:
asset_d = asset_dictionaries[1]

In [ ]:
asset_d['attributes']

In [ ]:
asset_d['connections']

In [ ]:
from asset_tracker.models import AssetTypeCode

In [ ]:
if {}: print('hey')

In [ ]:
import pickle
pickle.dumps(None)

In [ ]:
import pickle
pickle.dumps({})

In [ ]:
# Check if asset id exists in database for each asset
asset_id_by_temporary_id = {}

for asset_d in asset_dictionaries:
    asset_id = asset_d['id']
    try:
        asset_type_code = AssetTypeCode(asset_d['typeCode'])
    except ValueError:
        # !!! we have an unrecognized type code
        pass
    asset_name = asset_d['name']
    asset_attributes = asset_d.get('attributes', {})
    
    if asset_id in asset_id_by_temporary_id:
        asset_id = asset_id_by_temporary_id[asset_id]
    asset = db.query(Asset).get(asset_id)
    if not asset:
        asset = Asset.make_unique_record(db)
        asset_id_by_temporary_id[asset_id] = asset.id
        asset_id = asset.id
    asset.type_code = asset_type_code
    asset.name = asset_name
    asset.attributes = asset_attributes
    
    print(asset.id, asset.type_code, asset.name)
    db.add(asset)
# db.commit()

In [ ]:
asset_id_by_temporary_id

In [ ]:
asset.attributes

In [ ]:
# AssetTypeCode('x')

In [ ]:
asset.type_code.value

In [ ]:
db.query(Asset).count()

In [ ]:
# For each asset,
    # Save asset in database

In [ ]:
from invisibleroads_macros_security import make_random_string
make_random_string(10)

In [ ]:
from enum import Enum

class MyEnum(Enum):
    one = 1
    two = 2
    three = 3

MyEnum.one

In [ ]:
import enum
from sqlalchemy.types import Enum

class AssetTypeCode(enum.Enum):
    LINE = 'l'
    METER = 'm'
    TRANSFORMER = 't'
    SUBSTATION = 's'

In [ ]:
asset_d = asset_dictionaries[1]
asset_d['connections']

In [ ]:
db.query(Connection).get({'asset_id': 1, 'bus_id': 2})

In [ ]:
from asset_tracker.models import Bus, Connection

bus_id_by_temporary_id = {}

for asset_d in asset_dictionaries:
    asset_id = asset_d['id']
    if asset_id in asset_id_by_temporary_id:
        asset_id = asset_id_by_temporary_id[asset_id]
    asset = db.query(Asset).get(asset_id)
    for connection_d in asset_d.get('connections', []):
        bus_id = connection_d['busId']
        # If we recognize a temporary id,
        if bus_id in bus_id_by_temporary_id:
            # Use the permanent id
            bus_id = bus_id_by_temporary_id[bus_id] 
        bus = db.query(Bus).get(bus_id)
        if not bus:
            bus = Bus.make_unique_record(db)
            # Link temporary id to permanent id
            bus_id_by_temporary_id[bus_id] = bus.id
            bus_id = bus.id
        db.add(bus)
        connection = db.query(Connection).get({'asset_id': asset_id, 'bus_id': bus_id})
        if not connection:
            connection = Connection(asset_id=asset_id, bus_id=bus_id)
        connection.attributes = connection_d.get('attributes', {})
        db.add(connection)
        # print(bus_id, connection.attributes)

In [ ]:
for asset in db.query(Asset):
    print('!!!', asset.id, asset.attributes, asset.connections)
    for connection in asset.connections:
        print(connection.attributes)